# Agent
- Agent는 LLM과 여러 Tool을 조합하여 하나의 기능을 하는 애플리케이션을 쉽게 만들기 위한 프레임워크이다.
- [오픈소스 모델로 Agent 만들기 ](https://youtu.be/04MM0PXv2Fk)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "Agent Tutorial"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Tool
- Tool을 정의하여 LLM 추론에 같이 활용하도록 만들어져있다
- `Tavily Search`처럼 검색을 통해 최신 정보에 접근할 수 있게 하거나, python 코드를 실행하는 등의 다양한 Tool이 있다

### Tavily Search
- 검색을 통해 최신정보를 조회할 수 있는 Tool



In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=5)

In [4]:
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

[{'url': 'https://promenade7.tistory.com/entry/멋진-카카오-아지트-판교역-카카오-프렌즈샵-방문-후기',
  'content': "ì¹´ì¹´ì˜¤\xa0ìº�ë¦\xadí„°ê°€\xa0ì•„ë§ˆë�„\xa0LEDë¡œ\xa0ë³´ì�´ëŠ”\xa0ì\xa0„ê´‘íŒ�ì—�\xa0ê³„ì†�\xa0ë‚˜ì˜µë‹ˆë‹¤. ì�´ëŸ°\xa0ê±°\xa0ë§Œë“œëŠ”\xa0ì§�ì›�ë�„\xa0ì°¸\xa0ì‰½ì§€\xa0ì•Šê²\xa0ë„¤ìš”~\nê³³ê³³ì—�\xa0ìžˆëŠ”\xa0ì¹´ì¹´ì˜¤\xa0ìº�ë¦\xadí„°ì™€\xa0í•¨ê»˜\xa0ì‚¬ì§„ë�„\xa0ì°�ì�„\xa0ìˆ˜\xa0ìžˆìŠµë‹ˆë‹¤.\nì˜¬ë�¼ê°€ë³´ì§€ëŠ”\xa0ì•Šì•˜ì§€ë§Œ\xa0ìƒ�ì¸µë¶€ë¡œë�„\xa0ê³„ì†�\xa0ì´ˆë¡�ìƒ‰ë“¤ì�´\xa0ë³´ì�´ê³\xa0 1ì¸µì—�ë�„\xa0ì�´ë\xa0‡ê²Œ\xa0ë§ˆì¹˜\xa0ì™¸ë¶€\xa0ìž�ì—°ì�„\xa0ì˜®ê²¨\xa0ë†“ì�€\xa0ë“¯í•œ\xa0ì¡°ê²½ê³µê°„ì�´\xa0ë§Žì•˜ìŠµë‹ˆë‹¤. ì‚\xadë§‰í•\xa0\xa0ìˆ˜\xa0ìžˆëŠ”\xa0ë�„ì‹¬ì�˜\xa0ì˜¤í”¼ìŠ¤\xa0ë¹Œë”©\xa0ë‚´ë¶€ì—�\xa0ê·¸ëž˜ë�„\xa0ìž�ì—°ì�„\xa0ëŠ�ë‚„\xa0ìˆ˜\xa0ìžˆê²Œ\xa0ìž˜\xa0ë§Œë“¤ì–´ë†¨ë„¤ìš”.\n ì˜¤í”¼ìŠ¤ ê±´ë¬¼ì�´ë�¼ ë‹¹ì—°ížˆ ëª»ì˜¬ë�¼ê°€ê²\xa0ì£\xa0.ì•ˆê·¸ëž˜ë�„ íšŒì‹�ë‚\xa0ì�´ë‚˜ ì£¼ë³€ ì§€ë‚˜ê°€ë‹¤ ìž�ì£¼ íŒ�êµ� ì•„ì§€íŠ¸ ì˜†ì�„ ì§€ë‚˜ë‹¤ë‹ˆëŠ”ë�° ê±´ë¬¼\xa01ì¸µì—�\xa0ì¹´ì¹´ì˜¤í”„ë\xa0Œì¦ˆìƒµì�´\xa0ì

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

embeddings = OllamaEmbeddings(model="llama3")

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, embedding=embeddings)

# Retriever를 생성합니다.
retriever = vector.as_retriever()

- Tool의 description이 상당히 중요하다
- description에 따라 agent가 도구를 사용할 수도 있고 안할 수도 있다

In [4]:
# PDf 문서에서 Query 에 대한 관련성 높은 Chunk 를 가져옵니다.
print(retriever.get_relevant_documents(
    "YouTube의 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 알려줘"
)[0])

page_content='SPRi AI Brief |  \n2023-12 월호\n2G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의\nnG7이 첨단 AI 시스템을 개발하는 기업을 대상으로 AI 위험 식별과 완화를 위해 자발적인 \n채택을 권고하는 AI 국제 행동강령을 마련\nn행동강령은 AI 수명주기 전반에 걸친 위험 평가와 완화, 투명성과 책임성의 보장, 정보공유와 \n이해관계자 간 협력, 보안 통제, 콘텐츠 인증과 출처 확인 등의 조치를 요구KEY Contents\n£G7, 첨단 AI 시스템의 위험 관리를 위한 국제 행동강령 마련\nn주요 7개국(G7)* 은 2023 년 10월 30일 ‘히로시마 AI 프로세스 ’를 통해 AI 기업 대상의 AI 국제 \n행동강령 (International Code of Conduct for Advanced AI Systems) 에 합의\n∙G7은 2023 년 5월 일본 히로시마에서 개최된 정상회의에서 생성 AI에 관한 국제규범 마련과 \n정보공유를 위해 ‘히로시마 AI 프로세스 ’를 출범**\n∙기업의 자발적 채택을 위해 마련된 이번 행동강령은 기반모델과 생성 AI를 포함한 첨단 AI 시스템의 \n위험 식별과 완화에 필요한 조치를 포함\n* 주요 7개국(G7)은 미국, 일본, 독일, 영국, 프랑스 , 이탈리아 , 캐나다를 의미\n** 5월 정상회의에는 한국, 호주, 베트남 등을 포함한 8개국이 초청을 받았으나 , AI 국제 행동강령에는 우선 G7 국가만 포함하여 채택\nnG7은 행동강령을 통해 아래의 조치를 제시했으며 , 빠르게 발전하는 기술에 대응할 수 있도록 \n이해관계자 협의를 통해 필요에 따라 개정할 예정\n∙첨단 AI 시스템의 개발 과정에서 AI 수명주기 전반에 걸쳐 위험을 평가 및 완화하는 조치를 채택하고 , \n첨단 AI 시스템의 출시와 배포 이후 취약점과 오용 사고, 오용 유형을 파악해 완화\n∙첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알리는 방법으로 투

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


`create_retriever_tool`은 데이터를 검색하는 도구를 생성할 때 사용

In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="2023년 12월 AI 관련 정보를 PDF 문서에서 검색합니다. '2023년 12월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

- Agent가 사용할 tool 리스트 정의
- 이번 예제에서는 검색 Tool인 Tavily Search와 위에서 정의한 PDF 검색 Tool인 retriever_tool을 사용한다

In [6]:
tools = [search, retriever_tool]

In [12]:
from langchain_openai import ChatOpenAI
# from langchain_community.chat_models import ChatOllama
# model = ChatOllama(model="llama3")
model = ChatOpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model = "xionic-ko-llama-3-70b",
)

In [8]:
from langchain import hub

# hub에서 prompt를 가져옵니다 - 이 부분을 수정할 수 있습니다!
prompt = hub.pull("teddynote/react-chat-json-korean")

# prompt 의 messages를 출력합니다.
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

- 아래에서 생성한 Agent 인스턴스는 어떻게 Action을 취할 지 정의할 뿐 실제로 실행시키는 역할이 아니다
- OpenAI 모델이 아니면 되도록 `create_json_chat_agent`를 사용해서 agent를 정의해야 한다.
### [create_json_chat_agent](https://api.python.langchain.com/en/latest/agents/langchain.agents.json_chat.base.create_json_chat_agent.html)
- `create_json_chat_agent`는 prompt의 input_variables에 반드시 `tools`, `tools_name`, `agent_scratchpad`를 포함해야 한다
    1. `tools`: agent가 사용할 도구의 description과 arguments들이 들어간다
    1. `tools_name`: 사용할 모든 도구의 이름을 포함한다
    1. `agent_scratchpad`: 반드시 `MessagePlaceholder`이어야 하며, 이전 단계의 agent action과 tool output 메세지가 들어간다
#### [JSONAgentOutputParser](https://api.python.langchain.com/en/latest/agents/langchain.agents.output_parsers.json.JSONAgentOutputParser.html)
- `create_json_chat_agent` 내부적으로 `JSONAgentOutputParser`을 사용하고 있으며, 언어 모델 출력의 도구 호출과 최종 답변을 JSON 형식으로 구문 분석하도록 설계되었다
- 아래 2가지 형식의 output을 파싱한다
    1. 도구 호출, 이 형식을 파싱하여 `AgentAction` 객체를 도출한다
        ```json
        {
            "action": "tavily_search_results_json", // tool name
            "action_input": "2024년 아시안컵 대한민국의 축구 경기 결과" // input of tool
        }
        ``` 
        ```json
        {
            "output": {
                "tool": "tavily_search_results_json",
                "tool_input": "2024년 아시안컵 대한민국의 축구 경기 결과",
                "log": "```\n{\n    \"action\": \"tavily_search_results_json\",\n    \"action_input\": \"2024년 아시안컵 대한민국의 축구 경기 결과\"\n}\n```",
                "type": "AgentAction"
            }
        }
        ```
    1. 최종 답변, 이 형식을 파싱하여 `AgentFinish` 객체를 도출한다
        ```json
        {
            "action": "Final Answer",
            "action_input": "2024년 아시안컵 대한민국의 축구 경기 결과는 아직 확정되지 않았습니다. 현재까지의 경기 결과는 요르단에 0대2 충격패를 당했으며, 4강 결과는 아직 알려지지 않았습니다."
        }
        ```
        ```json
        {
            "output": {
                "return_values": {
                "output": "2024년 아시안컵 대한민국의 축구 경기 결과는 아직 확정되지 않았습니다. 현재까지의 경기 결과는 요르단에 0대2 충격패를 당했으며, 4강 결과는 아직 알려지지 않았습니다."
                },
                "log": "```\n{\n    \"action\": \"Final Answer\",\n    \"action_input\": \"2024년 아시안컵 대한민국의 축구 경기 결과는 아직 확정되지 않았습니다. 현재까지의 경기 결과는 요르단에 0대2 충격패를 당했으며, 4강 결과는 아직 알려지지 않았습니다.\"\n}\n```",
                "type": "AgentFinish"
            }
        }
        ```

In [13]:
from langchain.agents import create_json_chat_agent
agent = create_json_chat_agent(llm=model, tools=tools, prompt=prompt)

`AgentExecutor`는 Action을 정의한 Agent와 Tool을 활용하여 주어진 작업을 실행한다

In [14]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
response = agent_executor.invoke({"input": "2024년 아시안컵 대한민국의 축구 경기 결과를 알려줘."})
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "tavily_search_results_json",
    "action_input": "2024년 아시안컵 대한민국의 축구 경기 결과"
}
```[{'url': 'https://olympics.com/ko/news/2024-afc-u23-asian-cup-full-schedule-all-results-scores-and-standings-complete-list', 'content': '촬영 2021 Getty Images. 축구. 2024 아시아축구연맹 (AFC) U23 아시안컵에서는 남자 올림픽 축구 출전권 3장이 주어집니다. 4위 팀은 2024 파리 올림픽 대회 의 출전권을 놓고 U23 아프리카 예선 4위 팀인 기니를 상대로 플레이오프를 치르게 됩니다. 아시안컵 대회는 5월 3 ...'}, {'url': 'https://in.naver.com/itkwon/contents/internal/659218558880800', 'content': '2024 카타르 아시안컵 축구 4강 결과 대한민국 요르단에 0대2 충격패 결승좌절 남은경기와 결승전 일정. 2024. 2. 7. 3:07. 정말 아쉽게도 한국 축구팀의 4강 결과는 속도와 체력 기세 개인기 등 모든 요소에서 요르단에 부족했던 경기 였습니다. 역대 최강의 전력을 ...'}, {'url': 'https://www.heykakao.kr/2024/01/afc.html', 'content': '■ AFC 카타르 아시안컵 대한민국팀 엔트리\nFW\xa0 : 손흥민, 조규성\nMF : 정우영, 이강인, 이재성, 박용우, 홍현석, 황인범, 문선민, 황희찬, 양현준, 이순민, 박진섭, 오현규, 홍현석, 정우영\nDF : 박진섭, 정승현, 김민재, 김영권, 이기제, 설영우, 김태환, 김주성, 김진수, 정승현\nGK : 김승규, 조현우, 송범근\n댓글\n이 블로그의 인기 게시물\nKT 지니tv 채널번호 알아보기\n

In [16]:
# 'agent_executor' 객체의 'invoke' 메소드를 호출하여, 'langsmith'가 테스팅에 어떻게 도움을 줄 수 있는지에 대한 질문을 입력으로 제공합니다.
response = agent_executor.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    }
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "pdf_search",
    "action_input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화"
}
```2023 년 12월호

SPRi AI Brief |  
2023-12 월호
2G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의
nG7이 첨단 AI 시스템을 개발하는 기업을 대상으로 AI 위험 식별과 완화를 위해 자발적인 
채택을 권고하는 AI 국제 행동강령을 마련
n행동강령은 AI 수명주기 전반에 걸친 위험 평가와 완화, 투명성과 책임성의 보장, 정보공유와 
이해관계자 간 협력, 보안 통제, 콘텐츠 인증과 출처 확인 등의 조치를 요구KEY Contents
£G7, 첨단 AI 시스템의 위험 관리를 위한 국제 행동강령 마련
n주요 7개국(G7)* 은 2023 년 10월 30일 ‘히로시마 AI 프로세스 ’를 통해 AI 기업 대상의 AI 국제 
행동강령 (International Code of Conduct for Advanced AI Systems) 에 합의
∙G7은 2023 년 5월 일본 히로시마에서 개최된 정상회의에서 생성 AI에 관한 국제규범 마련과 
정보공유를 위해 ‘히로시마 AI 프로세스 ’를 출범**
∙기업의 자발적 채택을 위해 마련된 이번 행동강령은 기반모델과 생성 AI를 포함한 첨단 AI 시스템의 
위험 식별과 완화에 필요한 조치를 포함
* 주요 7개국(G7)은 미국, 일본, 독일, 영국, 프랑스 , 이탈리아 , 캐나다를 의미
** 5월 정상회의에는 한국, 호주, 베트남 등을 포함한 8개국이 초청을 받았으나 , AI 국제 행동강령에는 우선 G7 국가만 포함하여 채택
nG7은 행동강령을 통해 아래의 조치를 제시했으며 , 빠르게 발전하는 기술에 대응할 수 있도록 
이해관계자 협의를 통해 필요에 따라 개정할 예정
∙첨단 AI 시스템의 개발 과정에서 AI 수명주기 전반에 걸쳐 위험을 평가 및 완화하는